# Build homogeneous dataset

## Setting up environment

---



### Loading libraries

In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath(".."))

import pickle

from torch_geometric.data import Data
from box import Box

from util.postgres import create_sqlalchemy_engine
from util.homogeneous.dataset import DatasetEuCoHM, assert_bidirectional_edges

### Global variables

In [6]:
# -------------------- GLOBAL VARIABLES --------------------
PATH_TO_CONFIG_FILE = '../config.yaml'

# -------------------- LOAD CONFIGURATION --------------------
# Load the configuration file
config = Box.from_yaml(filename=PATH_TO_CONFIG_FILE)

num_train = 0.7             # Percentage of data used for training
num_bootstraping = 10
pg_engine = create_sqlalchemy_engine(
    username=config.POSTGRES.USERNAME,
    password=config.POSTGRES.PASSWORD,
    host=config.POSTGRES.HOST,
    port=config.POSTGRES.PORT,
    database=config.POSTGRES.DATABASE,
    schema=config.POSTGRES.SCHEMA
)

## Data preparation

---



In [4]:
def unit_testing(data):
    # Test: check that the number of elements in the positive edge index equals to the number of elements in the negative edge index
    assert data.test_pos_edge_index.numel() == data.test_neg_edge_index.numel()
    
    # Test: check that all positive edges are bidirectional
    assert_bidirectional_edges(edges=data.train_pos_edge_index)
    assert_bidirectional_edges(edges=data.test_pos_edge_index)
    print('All tests passed')

def save_dataset(dataset):
    dataset_save_filepath = f'../data/{dataset.get_dataset_name()}.pkl'
    # Before saving the dataset, we need to close the engine to connect to Postgres DB.
    dataset.close_engine()
    # Save the dataset
    with open(dataset_save_filepath, 'wb') as output:
        pickle.dump(dataset, output, pickle.HIGHEST_PROTOCOL)
        print(f'Dataset saved to {dataset_save_filepath}')

def build_dataset(use_periodical_embedding_decay: bool,
                  use_top_keywords: bool,
                  num_train: float, 
                  bootstrap_id: int):
    # Build the homogeneous graph
    data: Data
    author_node_id_map: dict
    author_id_map: dict
    dataset: DatasetEuCoHM = DatasetEuCoHM(
        pg_engine=pg_engine,
        num_train=num_train,
        use_periodical_embedding_decay=use_periodical_embedding_decay,
        use_top_keywords=use_top_keywords,
        bootstrap_id=bootstrap_id
    )
    data, author_node_id_map, author_id_map = dataset.build_homogeneous_graph()
    return dataset, data, author_node_id_map, author_id_map

In [ ]:
# Uncomment the ones you need to rebuild
dataset_configurations = [
    {'use_periodical_embedding_decay': False, 'use_top_keywords': False, 'num_train': num_train}
]

for conf in dataset_configurations:
    print(f'Processing dataset configuration {conf}...')
    # Building dataset
    dataset, data, author_node_id_map, author_id_map = build_dataset(
        use_periodical_embedding_decay=conf['use_periodical_embedding_decay'],
        use_top_keywords=conf['use_top_keywords'],
        num_train=conf['num_train']
    )
    # Unit testing
    unit_testing(data)

    # Save dataset
    save_dataset(dataset)

In [7]:
# Uncomment the ones you need to rebuild
dataset_configurations = [
    {'use_periodical_embedding_decay': False, 'use_top_keywords': False, 'num_train': num_train},
    # {'use_periodical_embedding_decay': True, 'use_top_keywords': False, 'num_train': num_train},
    # {'use_periodical_embedding_decay': False, 'use_top_keywords': True, 'num_train': num_train},
    # {'use_periodical_embedding_decay': True, 'use_top_keywords': True, 'num_train': num_train},
    # {'use_periodical_embedding_decay': True, 'use_top_keywords': False, 'num_train': 1.0},
]

for conf in dataset_configurations:
    print(f'Processing dataset configuration {conf}...')
    # Building dataset
    for bootstrap_id in range(num_bootstraping):
        dataset, data, author_node_id_map, author_id_map = build_dataset(
            use_periodical_embedding_decay=conf['use_periodical_embedding_decay'],
            use_top_keywords=conf['use_top_keywords'],
            num_train=conf['num_train'],
            bootstrap_id=bootstrap_id
        )
        # Unit testing
        unit_testing(data)
    
        # Save dataset
        save_dataset(dataset)

Processing dataset configuration {'use_periodical_embedding_decay': False, 'use_top_keywords': False, 'num_train': 0.7}...
Querying n-th time percentile...
Querying co-authorship edge data...
Querying author nodes...


AssertionError: The following edges are missing their reverse counterparts: [[50831, 3202], [49061, 35725], [23021, 39985], [50831, 51138], [30322, 47333], [39125, 10525], [46502, 34559], [15362, 20026], [43125, 41609], [19795, 59976], [27182, 11957], [20605, 30118], [20217, 36696], [43125, 32774], [9525, 33784], [14680, 53021], [22462, 9767], [8646, 585], [36395, 4262], [54571, 24561], [30118, 41665], [6621, 50203], [48440, 7645], [35636, 25066], [38049, 27505], [44499, 33830], [58033, 53254], [38259, 16823], [27521, 37183], [37107, 15724], [39154, 11467], [26808, 136], [53616, 286], [55093, 286], [53416, 286], [39047, 376], [5503, 437], [42110, 1094], [56557, 1335], [32239, 1498], [52114, 1512], [5052, 1570], [38221, 1578], [33384, 1583], [8712, 1915], [41418, 1915], [19471, 2065], [9140, 2099], [36212, 2099], [34651, 2099], [38318, 2099], [27963, 2152], [3932, 2322], [56062, 2470], [6790, 2568], [37039, 2592], [59545, 2630], [55108, 3116], [18091, 3259], [52653, 3259], [59929, 3382]